In [ ]:
from src.hpe.common.landmarks import MyLandmark
from src.hpe.mp.landmarks import can_predict

for landmark in MyLandmark:
    print(f"{landmark}: {can_predict(landmark)}")

In [ ]:
def func(pair):
    x, y = pair
    return x + y

xs = [(1, 4), (2, 5), (3, 6)]

zs = list(map(func, xs))

print(zs)

In [ ]:
from cv2.typing import MatLike
from typing import List, Any
from ultralytics import YOLO

from src.hpe.yolo.evaluate import predict_landmarks
from src.hpe.yolo.model import build_pose_model
from src.hpe.common.landmarks import YoloLabels, MyLandmark
from src.hpe.common.performance import AbstractPerformanceCollector

class EmptyLogger(AbstractPerformanceCollector):
    _model: YOLO | None = None

    def _process(self, image: MatLike, labels: List[YoloLabels]) -> Any:
        if self._model is None:
            self._model = build_pose_model()

        results = predict_landmarks(image, self._model)
        # print(f"Shape: {results._values.keypoints.shape}")
        # print(f"no_person_detected(): {results.no_person_detected()}")
        # print(f"keypoints missing: {results._values.keypoints.xyn.shape.count(0) > 0}")

        if (not results.no_person_detected()):
            print(results[MyLandmark.HEAD])
        

    def _post_process(self, name: str, results: List[Any]) -> Any:
        self._model = None

In [ ]:
logger = EmptyLogger()
logger.collect(name="")


In [1]:
from cv2.typing import MatLike
from typing import List, Any

from src.hpe.common.landmarks import YoloLabels, MyLandmark
from src.hpe.common.performance import AbstractPerformanceCollector
from src.hpe.mp.evaluate import predict_landmarks
from src.hpe.mp.model import build_holistic_model

class EmptyLogger(AbstractPerformanceCollector):

    def _process(self, image: MatLike, labels: List[YoloLabels]) -> Any:
        model = build_holistic_model()

        results = predict_landmarks(image, model)
        # print(f"Shape: {results._values.keypoints.shape}")
        # print(f"no_person_detected(): {results.no_person_detected()}")
        # print(f"keypoints missing: {results._values.keypoints.xyn.shape.count(0) > 0}")

        print(results[MyLandmark.HEAD])
        

    def _post_process(self, name: str, results: List[Any]) -> Any:
        pass

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

2025-10-02 14:51:11.472096: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


loaded labels


In [2]:
logger = EmptyLogger()
logger.collect(name="")

libEGL warning: DRI3: Screen seems not DRI3 capable
libEGL warning: DRI3: Screen seems not DRI3 capable
MESA: error: ZINK: failed to choose pdev
libEGL warning: egl: failed to create dri2 screen
I0000 00:00:1759409516.878004  320055 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1759409516.934312  323206 gl_context.cc:357] GL version: 3.1 (OpenGL ES 3.1 Mesa 24.2.8-1ubuntu1~24.04.1), renderer: D3D12 (NVIDIA TITAN RTX)
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1759409517.325658  323135 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1759409518.322502  323136 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1759409518.337201  323143 inference_feedback_manager.cc:114] Feedback manager requires a model with a single s

landmark {
  x: 0.777226388
  y: 0.14045319
  z: 0.0451498069
  visibility: 0.999798
}
landmark {
  x: 0.78815347
  y: 0.112098992
  z: 0.0303291064
  visibility: 0.999902725
}
landmark {
  x: 0.794361711
  y: 0.111374557
  z: 0.0301215518
  visibility: 0.999829292
}
landmark {
  x: 0.801345348
  y: 0.111119181
  z: 0.0301923808
  visibility: 0.999902248
}
landmark {
  x: 0.775961757
  y: 0.112751424
  z: 0.0567646399
  visibility: 0.999926925
}
landmark {
  x: 0.772877634
  y: 0.11220786
  z: 0.0563859418
  visibility: 0.999866247
}
landmark {
  x: 0.769248962
  y: 0.111673713
  z: 0.0562506318
  visibility: 0.999921441
}
landmark {
  x: 0.837470353
  y: 0.125784099
  z: 0.0167143624
  visibility: 0.999682307
}
landmark {
  x: 0.788256407
  y: 0.115642816
  z: 0.13517122
  visibility: 0.999579608
}
landmark {
  x: 0.792662382
  y: 0.166954339
  z: 0.0495476685
  visibility: 0.995142341
}
landmark {
  x: 0.778353035
  y: 0.163597763
  z: 0.0811337233
  visibility: 0.993771851
}
landmar

I0000 00:00:1759409518.773583  320055 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1759409518.801082  323284 gl_context.cc:357] GL version: 3.1 (OpenGL ES 3.1 Mesa 24.2.8-1ubuntu1~24.04.1), renderer: D3D12 (NVIDIA TITAN RTX)
W0000 00:00:1759409519.083828  323219 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1759409520.107433  323222 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1759409520.125580  323258 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1759409520.126673  323251 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:

landmark {
  x: 0.52009815
  y: 0.284886569
  z: -0.148190543
  visibility: 0.999924302
}
landmark {
  x: 0.529395342
  y: 0.270238549
  z: -0.203054622
  visibility: 0.999942303
}
landmark {
  x: 0.534191489
  y: 0.270914853
  z: -0.203414932
  visibility: 0.999912
}
landmark {
  x: 0.539290667
  y: 0.271535337
  z: -0.203106254
  visibility: 0.999944
}
landmark {
  x: 0.526502728
  y: 0.268203139
  z: -0.149718478
  visibility: 0.999966502
}
landmark {
  x: 0.528694391
  y: 0.267373323
  z: -0.150392398
  visibility: 0.999956369
}
landmark {
  x: 0.530711651
  y: 0.26644671
  z: -0.150630727
  visibility: 0.999972939
}
landmark {
  x: 0.581731617
  y: 0.273875266
  z: -0.33126989
  visibility: 0.999955893
}
landmark {
  x: 0.568676889
  y: 0.269889385
  z: -0.0885163322
  visibility: 0.999960303
}
landmark {
  x: 0.534378
  y: 0.303363711
  z: -0.17483668
  visibility: 0.9998
}
landmark {
  x: 0.530976832
  y: 0.301538169
  z: -0.107867673
  visibility: 0.999804795
}
landmark {
  x: 

I0000 00:00:1759409520.431368  320055 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1759409520.458648  323362 gl_context.cc:357] GL version: 3.1 (OpenGL ES 3.1 Mesa 24.2.8-1ubuntu1~24.04.1), renderer: D3D12 (NVIDIA TITAN RTX)
W0000 00:00:1759409520.733963  323297 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1759409521.764952  323299 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1759409521.782611  323297 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1759409521.783035  323336 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:

...

W0000 00:00:1759409573.740609  325782 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1759409574.766568  325784 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1759409574.782778  325812 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1759409574.783460  325806 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1759409574.788352  325828 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1759409574.799305  325837 inference_feedback_manager.cc:114] Feedback manager 

landmark {
  x: 0.495717764
  y: 0.802884817
  z: -0.373431891
  visibility: 0.999420285
}
landmark {
  x: 0.489157349
  y: 0.813003302
  z: -0.37604776
  visibility: 0.999531627
}
landmark {
  x: 0.486250401
  y: 0.814614
  z: -0.376354754
  visibility: 0.999458492
}
landmark {
  x: 0.484603763
  y: 0.814315856
  z: -0.376339942
  visibility: 0.999645233
}
landmark {
  x: 0.493944019
  y: 0.815066
  z: -0.371009976
  visibility: 0.99940145
}
landmark {
  x: 0.495721847
  y: 0.816992044
  z: -0.371271342
  visibility: 0.999308586
}
landmark {
  x: 0.497491628
  y: 0.818492532
  z: -0.371401638
  visibility: 0.999470294
}
landmark {
  x: 0.485159039
  y: 0.811196208
  z: -0.331218511
  visibility: 0.999525785
}
landmark {
  x: 0.50091207
  y: 0.814280629
  z: -0.311115026
  visibility: 0.99793303
}
landmark {
  x: 0.499623954
  y: 0.791092396
  z: -0.350643873
  visibility: 0.998414874
}
landmark {
  x: 0.503646433
  y: 0.795130432
  z: -0.346122503
  visibility: 0.998423219
}
landmark 

I0000 00:00:1759409575.265950  320055 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1759409575.282746  325922 gl_context.cc:357] GL version: 3.1 (OpenGL ES 3.1 Mesa 24.2.8-1ubuntu1~24.04.1), renderer: D3D12 (NVIDIA TITAN RTX)
W0000 00:00:1759409575.554732  325860 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


None


W0000 00:00:1759409576.566465  325859 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1759409576.581930  325892 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1759409576.583603  325871 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1759409576.586821  325890 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1759409576.594322  325878 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1759409576.600213  325862 inference_feedback_manager.cc:114] Feedback manager 

In [ ]:
from src.hpe.yolo.performance import read_distances as read_yolo_distances
from src.hpe.mp.performance import read_distances as read_mp_distances
from src.hpe.common.plot import plot_distances_boxplot

mp_dist = read_mp_distances(dataset_name="distances")
mp_bgr_dist = read_mp_distances(dataset_name="distances_bgr")

yolo_dist = read_yolo_distances(dataset_name="distances")
yolo_bgr_dist = read_yolo_distances(dataset_name="distances_bgr")

In [ ]:
plot_distances_boxplot(
    None,
    ("MediaPipe", mp_dist),
    ("MediaPipe on bgr", mp_bgr_dist),
    ("Yolov11", yolo_dist),
    ("Yolov11 on bgr", yolo_bgr_dist))

In [ ]:
plot_distances_boxplot(
    (0, 10),
    ("MediaPipe", mp_dist),
    ("MediaPipe on bgr", mp_bgr_dist),
    ("Yolov11", yolo_dist),
    ("Yolov11 on bgr", yolo_bgr_dist))

In [ ]:
from src.hpe_dnn.kfold import HpeDnnFoldCrossValidation
from src.sota.kfold import SOTAFoldCrossValidation
from src.common.plot import box_plot_accuracies

box_plot_accuracies(
    HpeDnnFoldCrossValidation.evaluation_instance("arch1-mp-kf"),
    HpeDnnFoldCrossValidation.evaluation_instance("arch1-mp-unbalanced-kf"),
    HpeDnnFoldCrossValidation.evaluation_instance("arch1-mp-dr0.3-kf"),
    HpeDnnFoldCrossValidation.evaluation_instance("arch1-yolo-kf"),
    # HpeDnnFoldCrossValidation.evaluation_instance("arch2-kf"), 
    # HpeDnnFoldCrossValidation.evaluation_instance("arch3-kf"), 
    # HpeDnnFoldCrossValidation.evaluation_instance("arch4-kf"), 
    # HpeDnnFoldCrossValidation.evaluation_instance("arch5-kf"),
    SOTAFoldCrossValidation.evaluation_instance("yolo11n-kf", "yolov11n-cls"))